<img src="./images/Banner_NB.png">

# Lesson 2: Web Based API

### Part A - Simple API Call (no parameters) - After Video 2.6

We will start with a very simple API example, retrieving the current position of the ISS. We will be using the [requests library](http://www.python-requests.org/en/latest/), which will handle our communication with the server.

In [ ]:
import requests 
# Make a get request to get the latest position of the international space station from the opennotify api.
response = requests.get("http://api.open-notify.org/iss-now.json")

response 

That looks good, we've received a response and it has status 200 - which means all was ok. 

Let's see what happens when we try to get a response with a wrong URL:

In [ ]:
response_try2 = requests.get("http://api.open-notify.org/iss")

response_try2

As we saw in the lecture, response 400 is response of an error (due to the wrong url we sent)

Let's look at the content of our previous, successful response:

In [ ]:
response.content

We can already see that this is JSON (though it is stored a `bytes` object), but we can check formally:

In [ ]:
response.headers['content-type']

In [ ]:
response.headers

We can decode this byte object, then the JSON will be readable. 

In [ ]:
response_j = response.content.decode("utf-8")
print(response_j)

In [ ]:
import json
response_d = json.loads(response_j)
print(type(response_d))
print(response_d)
response_d["iss_position"]

Let's take a look at the JSON here:




In [ ]:
response_d

This looks a lot like a dictionary! We have key-value pairs. 

We can use the [json library](https://docs.python.org/3/library/json.html) to convert JSON into objects:

Or, not surprisingly, pandas can also load a json object:

In [ ]:
import pandas as pd 

df = pd.read_json(response_j)
df

Look at the Dataframe we got, this isn't quite what we want - we probably want one row per timestamp and longitude and latitude as columns. For that we will introduce a helper function called `flatten` 

In [ ]:
def flatten(response_d):
    response_d["latitude"] = response_d["iss_position"]["latitude"]
    response_d["longitude"] = response_d["iss_position"]["longitude"]
    del(response_d["iss_position"])
    return response_d
flatten(response_d)



That looks better. Let's get a couple of positions of the ISS over time and save it as an array:

In [ ]:
import time

def pull_position():
    """Retreives the position of the ISS and returns it as a flat dictionary"""
    response = requests.get("http://api.open-notify.org/iss-now.json")
    response_j = response.content.decode("utf-8")
    response_d = json.loads(response_j)
    flat_response = flatten(response_d)
    return flat_response    

iss_position = []
    
# calls pull_position 10 times with 3 seconds break
for i in range(10):
    flat_response = pull_position()
    iss_position.append(flat_response)
    print(flat_response)
    time.sleep(3)
    
len(iss_position)

Now we can convert this into a nice dataframe:

In [ ]:
import pandas as pd

iss_position_df = pd.DataFrame(iss_position)
iss_position_df['timestamp']  = pd.to_datetime(iss_position_df['timestamp'], unit="s")

iss_position_df = iss_position_df.set_index(pd.DatetimeIndex(iss_position_df['timestamp']))
iss_position_df["latitude"] = iss_position_df["latitude"].map(float)
iss_position_df["longitude"] = iss_position_df["longitude"].map(float)
iss_position_df

Let's see how the ISS moves

In [ ]:
import matplotlib.pyplot as plt
# This next line tells jupyter to render the images inline
%matplotlib inline
plt.style.use('ggplot')
iss_position_df.plot(kind="scatter", x="latitude", y="longitude")
plt.show()


Now it's time to go and see video 2.7, continue to part B - once you completed that video..

### Part B - API Call with parameters - After Video 2.7

Requests can be parametrized. You can search for tweets of a specific user, for example, or [retrieve the time the ISS is over Holon](http://open-notify.org/Open-Notify-API/ISS-Pass-Times/)!

The way to query with a get request for the ISS is this:

`http://api.open-notify.org/iss-pass.json?lat=32.0158&lon=34.7874`

We, of course, could generate that URL ourselves, but the requests library is helpful here. Since JSON is similar to dictionaries, the requests library takes dictionaries for parameters.

In [ ]:
import requests
url = "http://api.open-notify.org/iss-pass.json"
coordinates = {"lat": 32.0158, "lon": 34.7874, "n":5}

r = requests.get(url, params=coordinates)
data = r.json()
print(data)
print(type(r))
print(type(data))

In [ ]:
import pandas as pd

#print(timestamp)
for i in range(5):
    timestamp = data["response"][i]["risetime"]
    print(pd.to_datetime(timestamp,  unit="s"))

So, the ISS will be above Holon this am!

## In class Exercise

Write a code in the cell below that will check when will the ISS be over your house (based on coordinates of your place/city)

In [ ]:
### Your code goes here.. 